In [2]:
%load_ext autoreload
%autoreload 2

In [19]:
import json, os
import pandas as pd
import numpy as np
from data_utils import *

Exception: Please set environment variable: fairness_data to point towards the data directory

In [15]:
results_dir="/Users/neelguha/Dropbox/NeelResearch/fairness/code/results/rate_constraints/adult-income/baseline/"
train_out = pd.read_pickle(os.path.join(results_dir, 'train_out'))
test_out = pd.read_pickle(os.path.join(results_dir, 'test_out'))

In [16]:
test_out.head()

,label,race_Black,race_White,gender_Female,gender_Male,predictions,predicted_class
0,0,1,0,0,1,-4.429726,False
1,0,0,1,0,1,-1.094755,False
2,1,0,1,0,1,-1.758171,False
3,1,1,0,0,1,0.435604,True
4,0,0,1,1,0,-3.992164,False


In [20]:
np.mean(test_out['label'] == test_out['predicted_class'])

0.8545543885510718